# Fourier Series
### MC Physics, Updated 03/23/23

Any function can be written as Fourier Series.

A Fourier Series is the representation of a periodic function (or equivalently a function defined over a finite domain) as a linear combination of sinusoids.  Let us call the period of the function $L$.

Since we can find the components of the fourier series by taking inner products (integrating), it is a mechanical procedure we can get a computer to perform for us. 


In [ ]:
# import useful packages

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# sympy declarations

x=sp.symbols('x',real=True)
L=sp.symbols('L',real=True,nonzero=True)
n,m=sp.symbols('n,m',integer=True)


##  Define Inner Product

Since it is so useful for determining fourier components, let's make a function to perform the inner product.

The inner product is the integral of the complex conjugate of the first function multiplied by the second function.  Sympy has functions for symbolic integration and complex conjugation.

The domain of integration is set inside the function, we will choose 0 to L for now.

In [ ]:
# inner product definition
def inner_product(f,g):
    # choose range, here 0 to L
    x0=0
    #L=1 #P=L=1
    return(sp.integrate(sp.conjugate(f)*g,(x, x0, L)))

## Choose Fourier Basis

We need to choose a set of sinusoids that will allow all $L$-period functions to be represented (a "complete basis").  There are several possibilities, but since I have repeatedly claimed that anything you can do with sines or cosines can be done easier with complex exponentials, I will choose complex exponentials (first) here.  

Notice that Sympy has definitions `sp.I` and `sp.pi` of $i$ and $\pi$.

In [ ]:
# choose general basis function, here complex exponential
def u(n):
    return(sp.exp((sp.I*2*sp.pi * n * x)/L))

In [ ]:
# checking by display
u(5)

In [ ]:
# checking for orthogonality

inner_product(u(4),u(5))

Ok, it worked once, let's try the general case.

In [ ]:
inner_product(u(n),u(m))

Those conditionals look ugly, let's see if they simplify.

In [ ]:
inner_product(u(n),u(m)).simplify()

### Check #1: Explain what functions are being used in the Fourier Series below

## Insert function to represent

Any function defined on our domain (here $0$ to $L$ was chosen) can be used.  I'll try the square of $x$.

In [ ]:
f= x**2
display(f)

#For piecewise functions:
#fp = sp.Piecewise((0, x < 0), (f, x <= 1), (0, True))
#display(f)

In [ ]:
sp.plot(f)

### Check #2: What function are we representing with a Fourier series, over what domain?

## Find Fourier coefficients (aka components)

Fourier's trick (assume Fourier series exists, take the inner product of the Fourier series and one of the basis functions, then solve) shows us that the Fourier component is the inner product of the function and the basis function divided by the inner product of the basis function with itself:

$$ c_n = \frac{\langle u_n,f \rangle}{\langle u_n,u_n \rangle} $$

In [ ]:
# coefficients
def c(n):
    return(inner_product(u(n),f)/inner_product(u(n),u(n)))


In [ ]:
c(3)

That looks a bit complicated, we should look to see if it can be simplified. Sympy may not find all useful simplifications, but it is easy to let it check for us.

In [ ]:
c(3).simplify()

Can Sympy give us a general answer?

In [ ]:
c(n).simplify()

Let's make sure it agrees with specific result in a few cases.

In [ ]:
c(7).simplify()

In [ ]:
c(0)

## Make a finite partial series

How will we check if this answer is correct?  We know the function it is supposed to match (between $0$ and $L$), so we should be able to compare. But how do we determine the value of this infinite series?  One good way is to plot a partial series where we terminate at some point.

In [ ]:
# partial series  (be careful with range, I am summing with n from -N to N here)
def S(N):
    return(sum(c(n)*u(n) for n in range(-N, N+1)))



In [ ]:
fapprox=S(5)
display(fapprox)

Not exactly the order I would write them in, but this looks ok.  Notice that Sympy has left terms proportional to $\pm \frac{i L^3}{\pi^3}$ that cancel.

In [ ]:
fapprox.simplify()

Not the nicest form, I would prefer smaller denominators.


In [ ]:
# partial series  (be careful with range, I am summing with n from -N to N here)
def S(N):
    return(sum(c(n).simplify()*u(n) for n in range(-N, N+1)))



In [ ]:
fapprox=S(5)
display(fapprox)

### Check #3 How many terms are included in the sum above? Think about how you would increase it (you'll need to do this later)

## Visualize the result

Let's visualize this series. (You will see an error, think about what it might be then keep reading with the next cell)

In [ ]:
sp.plot(fapprox,(x,0,L),show=False)

So that failed.  It is tough to tell from the error message that was returned, but the error stems from a variable remaining in the expression even though $x$ has been given a value.  One good thing to check is specific values that Sympy is trying to plot.  By using `.subs()` we can see a value of the function for a particular value of the independent variable.  Here I am setting the value of `x` to one.

In [ ]:
fapprox.subs(x,1)

This makes my mistake obvious: there is a variable $L$ remaining in the problem. We need to explicitly tell Sympy what $L$ to use. 

### Turn into a dimensionless problem

So computer math, even symbolic math, benefits from dimensionless variables.  If we measure $x$ in units of $L$, or equivalently use the ratio $x/L$ s our independent variable, this makes our independent variable dimensionless.  We can do this by setting the value of $L$ to one (so that $x=x/L$).  Notice that this also makes $f=x^2$ unitless (previously it had dimensions of length squared). 

In [ ]:
fapproxN=fapprox.subs(L,1)
display(fapproxN)

In [ ]:
sp.plot(fapproxN,(x,-2,2));

Let's plot both the Fourier partial series and the original function.

In [ ]:
sp.plot(fapproxN,(x,-2,2));
sp.plot(f,(x,-2,2));

It would be nice to compare these on the same scale, which requires a few steps: creating a python variable for the plot, extending that plot, and then showing the combined plot. 

In [ ]:
# f actual and approx
plot1=sp.plot(fapproxN,(x,-2,2),show=False)

plot1.extend(sp.plot(f, (x, -2, 2), line_color='r', show=False))
plot1.show();

### Check #4: Explain the graph above. Why does it look like that?

Notice the series matches the function reasonably well, but only in the domain we chose: $0$ to $L=1$.  Also notice that the series is periodic with period $L=1$.  

### Check #5: Change the code above (not directly above you'll have to go several cells up)  to increase the number of terms (at least double) and see how it improves the quality of the series approximation. What did you change?

### Check #6: Change the function from $x^2$ to something else and observe the series

## Extensions (If time allows)

### Alternate forms

Our function was purely real, but our partial series looks complex.  It isn't, the terms for $n$ and $-n$ add to give purely real functions (sines and cosines).  We could translate this ourselves, or we could let Sympy do it. 

In [ ]:
fapprox.expand(complex=True)

Interesting, Sympy writes the sines first, then the cosines, and the constant term last.

We could also chose a different set of basis functions ($u_n$'s) and find alternate Fourier series.

### Plot coefficients

In [ ]:
c(n).simplify()

In [ ]:
cN=c(n).subs(L,1).simplify()
display(cN)

In [ ]:
# f actual and approx
plot2=sp.plot(sp.re(cN),(n,-4,4),show=False)

plot2.extend(sp.plot(sp.im(cN),(n,-4,4), line_color='r', show=False))
plot2.show();

That doesn't look good.  Sympy is plotting coefficients like they are a continuous function, but they are discrete (with integer indices).  Since it is plotting between 0 and 1, the values blow up.  It also doesn't deal well with the special case at n=0.  One needs to use a different tool than Sympy to make discrete plots. Perhaps in this case by hand is easier: the real part is inversely proportional to n squared, and the imaginary part is inversely proportional to n. 

But let me try it anyway, you might learn a few tricks from the process.

In [ ]:
sp.re(c(n)).simplify().subs(L,1)

In [ ]:
sp.im(c(n)).simplify().subs(L,1)

In [ ]:
def cim(n):
    return(sp.im(c(n)).simplify().subs(L,1))

def cre(n):
    return(sp.re(c(n)).simplify().subs(L,1))


In [ ]:
cim(1)

In [ ]:
ind = np.linspace(-10, 10, 21)


In [ ]:
ind

In [ ]:
import numpy as np

cimn=sp.lambdify(n,cim(n),'numpy')
cren=sp.lambdify(n,cre(n),'numpy')

In [ ]:
cimn(3)

In [ ]:
plt.plot(ind,cren(ind),'.');
plt.plot(ind,cimn(ind),'.')

Notice that the (purely real) zero coefficient is largest, while the imaginary term is largest for other values since it is inversely proportional to n while the real part is inversely proportional to n squared. 